In [2]:
import numpy as np
import pandas as pd

In [3]:
df = pd.read_csv("base_projet_teleco.csv", sep = ";")
df.drop("Unnamed: 0", axis = 1, inplace = True)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49071 entries, 0 to 49070
Data columns (total 77 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   CONTRACT_KEY              49071 non-null  float64
 1   CUSTOMER_AGE              49071 non-null  float64
 2   CUSTOMER_GENDER           49071 non-null  object 
 3   CONTRACT_TENURE_DAYS      49071 non-null  float64
 4   AVERAGE_CHARGE_6M         49061 non-null  float64
 5   FAILED_RECHARGE_6M        49071 non-null  float64
 6   AVERAGE_RECHARGE_TIME_6M  49060 non-null  float64
 7   BALANCE_M3                49071 non-null  float64
 8   BALANCE_M2                49071 non-null  float64
 9   BALANCE_M1                49071 non-null  float64
 10  FIRST_RECHARGE_VALUE      49068 non-null  float64
 11  LAST_RECHARGE_VALUE       49068 non-null  float64
 12  TIME_TO_GRACE             49068 non-null  float64
 13  TIME_TO_AFTERGRACE        49068 non-null  float64
 14  RECENC

In [5]:
# les données sont figés à fin M1 (août), M2 = juillet etc jusqu'à M6 = mars

# création de 4 variables (comme on cherche les churners dans les 2 mois)

df["FLAG_RECHARGE_M1"] = df["RECENCY_OF_LAST_RECHARGE"].apply(lambda x : 1 if 0 <= x <= 31 else 0)

df["FLAG_RECHARGE_M2"] = df["RECENCY_OF_LAST_RECHARGE"].apply(lambda x : 1 if 32 <= x <= 62 else 0)

df["FLAG_RECHARGE_M3"] = df["RECENCY_OF_LAST_RECHARGE"].apply(lambda x : 1 if 63 <= x <= 92 else 0)

df["FLAG_RECHARGE_PLUS_M3"] = df["RECENCY_OF_LAST_RECHARGE"].apply(lambda x : 1 if x >= 93 else 0) #plus loin que M3


In [6]:
# montant de recharge par mois 

# on a le flag du mois de la dernière recharge, le nombre de recharge sur 6 mois, le montant de la dernière recharge
# le délais inter-recharge et l'ancienneté du contrat

df.iloc[0].loc["NO_OF_RECHARGES_6M"] #exemple ici on a 7 recharge sur les 6 derniers mois
df.iloc[0].loc["AVERAGE_RECHARGE_TIME_6M"] #on a 27j entre les recharges
df.iloc[0].loc["FLAG_RECHARGE_M1"] #dernière recharge en M1
df.iloc[0].loc["LAST_RECHARGE_VALUE"] # dernière recharge vaut 50
df.iloc[0].loc["TOTAL_RECHARGE_6M"] #valeur de 400 au total donc sur M1 on a 50 donc il nous reste 350
df.iloc[0].loc["AVERAGE_CHARGE_6M"] #moyenne de recharge sur les 6 mois : 66.67
df.iloc[0].loc["CONTRACT_TENURE_DAYS"]  #contrat date de 1735 jours donc valeur à l'activation pas dans le 6 mois

# donc sur les 5 autres mois (M2 et +) on a 6 recharge faite pour un montant de 350 et 27 jours entre chaque
# quelles sont les combinaisons possibles ? 
  # -> impossible de faire une recharge de 200 et de 20
  # -> reste 50 et 100 qui peuvent être répartis de plusieurs façon possible (peut-être voir avec la balance)


# me paraît très compliquer à mettre en place


1735.0

In [7]:
# sinon approche via les balances : si balance M1 > M2 ou balance M2 > M3 ou balance M1 > M3 alors il a eu plusieurs recharges sur les 3 mois
# marche que si balance = reste des recharges 

for index, row in df.iterrows():
    if row["BALANCE_M1"] > row["BALANCE_M2"] :
        df.at[index, "AVERAGE_MULTIPLE_RECHARGE_M1_M2_M3"] = 1
    
    elif row["BALANCE_M2"] > row["BALANCE_M3"] :
        df.at[index, "AVERAGE_MULTIPLE_RECHARGE_M1_M2_M3"] = 1

    elif row["BALANCE_M1"] > row["BALANCE_M3"] :
        df.at[index, "AVERAGE_MULTIPLE_RECHARGE_M1_M2_M3"] = 1

    else :
        df.at[index, "AVERAGE_MULTIPLE_RECHARGE_M1_M2_M3"] = 0

In [8]:
df.iloc[0]

CONTRACT_KEY                             46.0
CUSTOMER_AGE                             49.0
CUSTOMER_GENDER                       b'Male'
CONTRACT_TENURE_DAYS                   1735.0
AVERAGE_CHARGE_6M                       66.67
                                       ...   
FLAG_RECHARGE_M1                            1
FLAG_RECHARGE_M2                            0
FLAG_RECHARGE_M3                            0
FLAG_RECHARGE_PLUS_M3                       0
AVERAGE_MULTIPLE_RECHARGE_M1_M2_M3        1.0
Name: 0, Length: 82, dtype: object

In [9]:
# si quelque chose entre 1 sinon 0

for index, row in df.iterrows() :
    if row["INC_DURATION_MINS_M1"] + row["INC_PROP_SMS_CALLS_M1"] == 0 :
        df.at[index, "FLAG_IN_M1"] = 0

    if row["INC_DURATION_MINS_M2"] + row["INC_PROP_SMS_CALLS_M2"] == 0 :
        df.at[index, "FLAG_IN_M2"] = 0

    
    if row["INC_DURATION_MINS_M3"] + row["INC_PROP_SMS_CALLS_M3"] == 0 :
        df.at[index, "FLAG_IN_M3"] = 0


    else :
        df.at[index, "FLAG_IN_M1"] = 1
        df.at[index, "FLAG_IN_M2"] = 1
        df.at[index, "FLAG_IN_M3"] = 1

In [10]:
# si quelque chose sort 1 sinon 0

for index, row in df.iterrows() :
    if row["OUT_DURATION_MINS_M1"] + row["OUT_SMS_NO_M1"] == 0 :
        df.at[index, "FLAG_OUT_M1"] = 0

    if row["OUT_DURATION_MINS_M2"] + row["OUT_SMS_NO_M2"] == 0 :
        df.at[index, "FLAG_OUT_M2"] = 0

    
    if row["OUT_DURATION_MINS_M3"] + row["OUT_SMS_NO_M3"] == 0 :
        df.at[index, "FLAG_OUT_M3"] = 0


    else :
        df.at[index, "FLAG_OUT_M1"] = 1
        df.at[index, "FLAG_OUT_M2"] = 1
        df.at[index, "FLAG_OUT_M3"] = 1

In [11]:
for index, row in df.iterrows() :
    if row["CONTRACT_TENURE_DAYS"] > 365 :
        row["OLD_CONTRACT"] = 1
    
    else :
        row["OLD_CONTRACT"] = 0

In [12]:
df.CONTRACT_TENURE_DAYS.sort_values(ascending=False)

109      37862.0
46850    37862.0
46013    37862.0
57        1740.0
277       1740.0
          ...   
48983      183.0
49046      183.0
49047      183.0
48538      183.0
48869      118.0
Name: CONTRACT_TENURE_DAYS, Length: 49071, dtype: float64

In [13]:
df

,CONTRACT_KEY,CUSTOMER_AGE,CUSTOMER_GENDER,CONTRACT_TENURE_DAYS,AVERAGE_CHARGE_6M,FAILED_RECHARGE_6M,AVERAGE_RECHARGE_TIME_6M,BALANCE_M3,BALANCE_M2,BALANCE_M1,...,FLAG_RECHARGE_M2,FLAG_RECHARGE_M3,FLAG_RECHARGE_PLUS_M3,AVERAGE_MULTIPLE_RECHARGE_M1_M2_M3,FLAG_IN_M1,FLAG_IN_M2,FLAG_IN_M3,FLAG_OUT_M1,FLAG_OUT_M2,FLAG_OUT_M3
0,46.0,49.0,b'Male',1735.0,66.67,1.0,27.0,15.83,8.83,23.83,...,0,0,0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
1,47.0,99.0,b'Male',1735.0,33.33,0.0,97.0,284.13,207.83,178.13,...,0,1,0,0.0,1.0,1.0,1.0,1.0,1.0,1.0
2,55.0,99.0,b'Male',1734.0,50.00,0.0,36.0,24.72,46.77,1.87,...,1,0,0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
3,62.0,50.0,b'Male',1724.0,16.67,0.0,99.0,21.53,89.43,70.43,...,1,0,0,1.0,1.0,1.0,1.0,NaN,NaN,0.0
4,148.0,22.0,b'Male',1735.0,25.00,0.0,18.0,28.84,8.54,8.39,...,0,0,0,0.0,NaN,NaN,0.0,NaN,NaN,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49066,3216840.0,51.0,b'Unknown',185.0,16.67,0.0,119.0,77.00,45.30,21.30,...,0,1,0,0.0,1.0,1.0,1.0,1.0,1.0,1.0
49067,3218562.0,99.0,b'Male',192.0,33.33,0.0,36.0,0.25,5.45,3.35,...,0,0,0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
49068,3251653.0,34.0,b'Male',184.0,16.67,0.0,59.0,2.80,77.80,73.60,...,1,0,0,1.0,NaN,NaN,0.0,NaN,NaN,0.0
49069,3255790.0,66.0,b'Male',186.0,33.33,0.0,102.0,5.33,0.33,0.33,...,0,1,0,0.0,1.0,1.0,1.0,1.0,1.0,1.0
